In [1]:
# To manipulate data
import pandas as pd

# Plotting librairies
import matplotlib.pyplot as plt
import networkx as nx
import matplotlib as mpl

In [2]:
from joblib import load
import os

# Define the parent directory
parent_dir = os.path.abspath(
    os.path.join(os.getcwd(), os.pardir, os.pardir)
)

# Define the subfolder directory
output_folder = os.path.join(
    parent_dir, "outputs", "tests_outputs"
)

# Name of the model output file
file_name1 = "BayesNet-fit-test3.joblib"
file_name2 = "BayesNet-IndTests-test3.joblib"

model = load(
    os.path.join(
        output_folder,
        file_name1
    )
)

tests_df = load(
    os.path.join(
        output_folder,
        file_name2
    )
)

## Approximate inference

In [3]:
from pgmpy.inference import ApproxInference

infer = ApproxInference(model)

In [9]:
infer.query(
    variables=["hunting_success", "pred_speed_bin"],
    joint=True,
    n_samples=50000,
    seed = 123
)

  0%|          | 0/11 [00:00<?, ?it/s]

<DiscreteFactor representing phi(hunting_success:5, pred_speed_bin:5) at 0x728382c8fcd0>

In [15]:
from pgmpy.inference import CausalInference

infer_adjusted = CausalInference(model)
print(infer_adjusted.query(variables=["hunting_success"], do={"pred_speed_bin": }))
#print(infer_adjusted.query(variables=["hunting_success"], do={"pred_speed_bin": 0}))

SyntaxError: closing parenthesis ']' does not match opening parenthesis '(' (2365859996.py, line 4)

In [27]:
print(f'Check model: {model.check_model()}\n')
for cpd in model.get_cpds():
    print(f'CPT of {cpd.variable}:')
    print(cpd, '\n')

Check model: True

CPT of prior_prey_speed_bin:
+--------------------------------------+----------+
| prior_prey_speed_bin((0.619, 2.153]) | 0.192479 |
+--------------------------------------+----------+
| prior_prey_speed_bin((2.153, 2.35])  | 0.196631 |
+--------------------------------------+----------+
| prior_prey_speed_bin((2.35, 2.506])  | 0.199555 |
+--------------------------------------+----------+
| prior_prey_speed_bin((2.506, 2.672]) | 0.205069 |
+--------------------------------------+----------+
| prior_prey_speed_bin((2.672, 3.664]) | 0.206265 |
+--------------------------------------+----------+ 

CPT of prior_chase_count_bin:
+-----+
| ... |
+-----+
| ... |
+-----+
| ... |
+-----+
| ... |
+-----+
| ... |
+-----+
| ... |
+-----+ 

CPT of prior_success:
+-----------------------+-----+
| prior_chase_count_bin | ... |
+-----------------------+-----+
| prior_success(0)      | ... |
+-----------------------+-----+
| prior_success(1)      | ... |
+-----------------------+---